# Primera parte: Clasificación de textos

En esta parte, utilizaremos **embeddings** para resolver un problema de clasificación de texto. Los embeddings, representaciones distribuidas y vectoriales de elementos, son un concepto muy común en el mundo del deep learning. Los **word vectors** que hemos visto en clase son una representación en forma de embedding de las palabras.

Para realizar este trabajo y sacarle el máximo partido, se recomienda ver los siguientes vídeos de clase:

*   Clasificación de texto con Word Vectors.
*   Análisis de overfitting con un modelo bag of words.
*   Clasificación de texto con RNN

Vamos a utilizar el dataset **"Reuters newswire topics classification"**, disponible desde Keras de manera similar al dataset de IMDB ([ver documentación](https://keras.io/datasets/#reuters-newswire-topics-classification)).

---

Se pide:

Entrenar un modelo **utilizando embeddings** que consiga un **65% de accuracy en test (55% si usamos RNNs)**, mostrando el entrenamiento y el resultado final.
 
Tenemos varias opciones para entrenar modelos con embeddings. El alumno puede explorar más de una pero es suficiente con conseguir un modelo que alcance la accuracy requerida:

*   Utilizar una **media de embeddings** al estilo de lo visto en el vídeo *Clasificación de texto con Word Vectors*
*   Utilizar una **CNN** sobre una secuencia de word vectors. Aquí necesitamos cambiar un poco la idea de convolución para actuar sobre sequencias de vectores. Keras incluye una [Convolución en 1D](https://keras.io/layers/convolutional/#conv1d) que puede ser utilizada en este caso, con un ejemplo de uso en la documentación. Una forma de hacer funcionar este esquema sería utilizar la convolución en 1D + max pooling.
*  Utilizar una **RNN** sobre una secuencia de word vectors, al estilo de lo visto en el vídeo *Clasificación de texto con RNN*. Para este problema es un poco complicado conseguir un buen modelo con RNNs, y además es más difícil experimentar ya que las redes recurrentes son modelos lentos de entrenar. Por eso, es suficiente con alcanzar un 55% de accuracy si optamos por utilizar un modelo de este estilo. Un buen consejo es emplear una red recurrente bidireccional como se ve en el vídeo *Clasificación de texto con RNN*.



---

Dos hiperparámetros importantes a elegir en el modelo son la **longitud de las secuencias de texto** y el **tamaño del vocabulario** para los embeddings. Podéis experimentar con ambos, o utilizar los mismos que se usan en los vídeos. Nótese que, al cortar todas las secuencias para que tengan el mismo tamaño, podríamos estar perdiendo mucho texto si elegimos un tamaño de secuencia demasiado pequeño. Igualmente, si las hacemos muy largas necesitaremos más tiempo para entrenar nuestros modelos. Una buena idea consiste en explorar los datos para ver cómo suelen ser de largos los textos y encontrar un buen trade-off para el tamaño de al secuencia.



---

Los embeddings que hemos visto en los vídeos se entrenan junto al modelo.  Una técnica frecuente es inicializar estos embeddings con word-vectors pre-entrenados en un gran corpus de texto, como hemos visto en clase. Esto puede ayudar ya que nuestro modelo empieza con unos embeddings que ya encapsulan significado. Si bien no es necesario para esta práctica, podéis ver cómo usar esta técnica [en el siguiente tutorial](https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html).




In [1]:
## Tu código

# Word Vectors

In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt


from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding, Bidirectional, Dropout, GlobalAveragePooling1D
from keras.layers import LSTM
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.datasets import reuters

Using TensorFlow backend.


In [3]:
print(tf.__version__)

2.1.0


In [4]:
maxlen = 100 #Crea arreglos de máximo este largo
batch_size = 10
max_features = 10000 #corta a este número de palabras más importantes

In [5]:
#Separamos el conjunto de train y de test
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words = max_features)

Comprobación de tamaños de conjuntos de train

In [6]:
print("Training entries: {}, labels: {}".format(len(x_train), len(y_train)))

Training entries: 8982, labels: 8982


Revisión de cuántos elementos o categorías tiene el dataset

In [7]:
max(y_train)

45

Indica la cantidad de categorías de clasificación, irían de 0 -- 45

Por lo tanto se cuenta con 46 capas

Es necesario hacer una codificación one hot a las categorías de 'y'

In [8]:
print("Elementos sin codificar:",y_train[0],' ', y_test[5])
from keras.utils import to_categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
print("Elementos codificados:",y_train[0],' ', y_test[5])

Elementos sin codificar: 3   3
Elementos codificados: [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]   [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [9]:
print(x_train[0])

[1, 2, 2, 8, 43, 10, 447, 5, 25, 207, 270, 5, 3095, 111, 16, 369, 186, 90, 67, 7, 89, 5, 19, 102, 6, 19, 124, 15, 90, 67, 84, 22, 482, 26, 7, 48, 4, 49, 8, 864, 39, 209, 154, 6, 151, 6, 83, 11, 15, 22, 155, 11, 15, 7, 48, 9, 4579, 1005, 504, 6, 258, 6, 272, 11, 15, 22, 134, 44, 11, 15, 16, 8, 197, 1245, 90, 67, 52, 29, 209, 30, 32, 132, 6, 109, 15, 17, 12]


In [10]:
len(x_train[0]), len(x_train[3])

(87, 224)

In [11]:
#A dictionary mapping words to an integer index
word_index = reuters.get_word_index()

#The first indices are reserved
word_index = {k:(v+3) for k,v in word_index.items()}
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2 #Unknown
word_index["<UNUSED>"] = 3

reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
  return ' '.join([reverse_word_index.get(i, '?') for i in text])

In [12]:
decode_review(x_train[3])

"<START> the farmers home administration the u s agriculture department's farm lending arm could lose about seven billion dlrs in outstanding principal on its severely <UNK> borrowers or about one fourth of its farm loan portfolio the general accounting office gao said in remarks prepared for delivery to the senate agriculture committee brian crowley senior associate director of gao also said that a preliminary analysis of proposed changes in <UNK> financial eligibility standards indicated as many as one half of <UNK> borrowers who received new loans from the agency in 1986 would be <UNK> under the proposed system the agency has proposed evaluating <UNK> credit using a variety of financial ratios instead of relying solely on <UNK> ability senate agriculture committee chairman patrick leahy d vt <UNK> the proposed eligibility changes telling <UNK> administrator <UNK> clark at a hearing that they would mark a dramatic shift in the agency's purpose away from being farmers' lender of last 

In [13]:
#Se agregan los PAD a la data de test y de entrenamiento
train_data = keras.preprocessing.sequence.pad_sequences(x_train,
                                                        value=word_index["<PAD>"],
                                                        padding='post',
                                                        maxlen=maxlen)

test_data = keras.preprocessing.sequence.pad_sequences(x_test,
                                                        value=word_index["<PAD>"],
                                                        padding='post',
                                                        maxlen=maxlen)

Comprobación de tamaños y contenido de los conjuntos

In [14]:
len(train_data[0]), len(train_data[1])

(100, 100)

In [15]:
print(train_data[0])

[   1    2    2    8   43   10  447    5   25  207  270    5 3095  111
   16  369  186   90   67    7   89    5   19  102    6   19  124   15
   90   67   84   22  482   26    7   48    4   49    8  864   39  209
  154    6  151    6   83   11   15   22  155   11   15    7   48    9
 4579 1005  504    6  258    6  272   11   15   22  134   44   11   15
   16    8  197 1245   90   67   52   29  209   30   32  132    6  109
   15   17   12    0    0    0    0    0    0    0    0    0    0    0
    0    0]


In [16]:
x_val = train_data[:4000]
partial_x_train = train_data[4000:]

y_val = y_train[:4000]
partial_y_train = y_train[4000:]

## Embedding

In [17]:
#Creación del modelo de embeddings con capa de dropout para regularizar la red
#Y una capa de average pooling para simplificar los modelos de embedding utilizados
vocab_size = max_features

model_embed = Sequential()
model_embed.add(Embedding(vocab_size, 16))
model_embed.add(Dropout(0.2))
model_embed.add(GlobalAveragePooling1D())
model_embed.add(Dense(46, activation=tf.nn.sigmoid))
#Se utilizan 46 capas de activación por la cantidad de categorías del dataset

model_embed.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 16)          160000    
_________________________________________________________________
dropout_1 (Dropout)          (None, None, 16)          0         
_________________________________________________________________
global_average_pooling1d_1 ( (None, 16)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 46)                782       
Total params: 160,782
Trainable params: 160,782
Non-trainable params: 0
_________________________________________________________________


In [18]:
#Se utiliza una métrica de pérdida de categorical crossentropy
#Esto por la cantidad de categorías que tiene el dataset
model_embed.compile(optimizer="Adam",
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [19]:
len(x_val), len(partial_x_train)

(4000, 4982)

In [20]:
history_embed = model_embed.fit(partial_x_train,
                    partial_y_train,
                    epochs = 40,
                    batch_size = batch_size,
                    validation_data=(x_val, y_val),
                    verbose=1)

E:\Users\NWUSER\anaconda3\envs\tf\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 4982 samples, validate on 4000 samples
Epoch 1/40
4982/4982 [==============================] - 2s 357us/step - loss: 3.2191 - accuracy: 0.2547 - val_loss: 2.5715 - val_accuracy: 0.2122
Epoch 2/40
4982/4982 [==============================] - 1s 172us/step - loss: 2.3206 - accuracy: 0.4049 - val_loss: 2.2813 - val_accuracy: 0.4367
Epoch 3/40
4982/4982 [==============================] - 1s 165us/step - loss: 2.1872 - accuracy: 0.4289 - val_loss: 2.2185 - val_accuracy: 0.4173
Epoch 4/40
4982/4982 [==============================] - 1s 174us/step - loss: 2.0769 - accuracy: 0.3864 - val_loss: 2.0566 - val_accuracy: 0.4162
Epoch 5/40
4982/4982 [==============================] - 1s 176us/step - loss: 1.9321 - accuracy: 0.4398 - val_loss: 1.9675 - val_accuracy: 0.4415
Epoch 6/40
4982/4982 [==============================] - 1s 182us/step - loss: 1.8533 - accuracy: 0.4613 - val_loss: 1.9074 - val_accuracy: 0.4625
Epoch 7/40
4982/4982 [==============================] - 1s 168us/step - loss

In [21]:
results = model_embed.evaluate(test_data, y_test)

print(results)

2246/2246 [==============================] - 0s 13us/step
[1.1147394734414051, 0.7324131727218628]


**Se alcanza un accuracy de 73.24% en los datos de test, que son datos completamente desconocidos por lo tanto es un resultado aceptable.**

A continuación se realizan pruebas con otras arquitecturas utilizando embeddings

# CNN

In [22]:
#Arquitectura de red convolucional de 1D que pueda reconocer 
#los PADS en los embeddings y pueda entrenarse adecuadamente

vocab_size = max_features
#Filtros de convolución y tamaño de kernel de recorrido en los
#embeddings, se agregan capas de dropout  y al final una capa
#Dense (Además de la final de clasificación) que permita simplificar
#las salidas de la capa de convolución

filters = 250
kernel_size = 3

model_cnn = Sequential()
model_cnn.add(Embedding(vocab_size, 16))
model_cnn.add(Dropout(0.2))
model_cnn.add(Conv1D(filters,
                    kernel_size,
                    padding='valid',
                    activation='relu',
                    strides=1))
model_cnn.add(GlobalMaxPooling1D())
model_cnn.add(Dense(250, activation=tf.nn.relu))
model_cnn.add(Dropout(0.2))
model_cnn.add(Dense(46, activation=tf.nn.sigmoid))

model_cnn.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 16)          160000    
_________________________________________________________________
dropout_2 (Dropout)          (None, None, 16)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, None, 250)         12250     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 250)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 250)               62750     
_________________________________________________________________
dropout_3 (Dropout)          (None, 250)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 46)               

In [23]:
model_cnn.compile(optimizer="Adam",
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [24]:
history_cnn = model_cnn.fit(partial_x_train,
                    partial_y_train,
                    epochs = 6, #Se limita a 6 épocas por regularización con Early Stopping
                    batch_size = batch_size,
                    validation_data=(x_val, y_val),
                    verbose=1)

E:\Users\NWUSER\anaconda3\envs\tf\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 4982 samples, validate on 4000 samples
Epoch 1/6
4982/4982 [==============================] - 2s 358us/step - loss: 2.2278 - accuracy: 0.3695 - val_loss: 1.8356 - val_accuracy: 0.4950
Epoch 2/6
4982/4982 [==============================] - 2s 322us/step - loss: 1.6244 - accuracy: 0.5813 - val_loss: 1.5480 - val_accuracy: 0.6325
Epoch 3/6
4982/4982 [==============================] - 2s 314us/step - loss: 1.3452 - accuracy: 0.6712 - val_loss: 1.4096 - val_accuracy: 0.6660
Epoch 4/6
4982/4982 [==============================] - 2s 306us/step - loss: 1.1439 - accuracy: 0.7077 - val_loss: 1.3427 - val_accuracy: 0.6790
Epoch 5/6
4982/4982 [==============================] - 2s 321us/step - loss: 0.9438 - accuracy: 0.7551 - val_loss: 1.3235 - val_accuracy: 0.7032
Epoch 6/6
4982/4982 [==============================] - 2s 314us/step - loss: 0.7740 - accuracy: 0.7941 - val_loss: 1.3420 - val_accuracy: 0.7020


In [25]:
results = model_cnn.evaluate(test_data, y_test)

print(results)

2246/2246 [==============================] - 0s 109us/step
[1.3155283704997172, 0.6932324171066284]


**La red convolucional alcanza un accuracy de 69.32%, sigue siendo adecuado pero es considerablemente menor que la red de embeddings simple, pero no se entrenan más épocas por la tendencia a caer en overfitting**

# RNN

In [26]:
#Se construye una red  bidireccional simple que reciba los enbeddings
model_rnn =  Sequential()
model_rnn.add(Embedding(max_features, 128))
model_rnn.add(Bidirectional(LSTM(16)))
model_rnn.add(Dropout(0.5))
model_rnn.add(Dense(46, activation='sigmoid'))

model_rnn.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, None, 128)         1280000   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 32)                18560     
_________________________________________________________________
dropout_4 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 46)                1518      
Total params: 1,300,078
Trainable params: 1,300,078
Non-trainable params: 0
_________________________________________________________________


In [27]:
model_rnn.compile(optimizer="Adam",
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [28]:
history_rnn = model_rnn.fit(partial_x_train,
                    partial_y_train,
                    epochs = 20, ##Se limita a 20 épocas por regularización con Early Stopping
                    batch_size = batch_size,
                    validation_data=(x_val, y_val),
                    verbose=1)

E:\Users\NWUSER\anaconda3\envs\tf\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 4982 samples, validate on 4000 samples
Epoch 1/20
4982/4982 [==============================] - 23s 5ms/step - loss: 2.6900 - accuracy: 0.3448 - val_loss: 2.2527 - val_accuracy: 0.3483
Epoch 2/20
4982/4982 [==============================] - 22s 4ms/step - loss: 2.1300 - accuracy: 0.3743 - val_loss: 2.0048 - val_accuracy: 0.4132
Epoch 3/20
4982/4982 [==============================] - 23s 5ms/step - loss: 1.9219 - accuracy: 0.4255 - val_loss: 1.8992 - val_accuracy: 0.4340
Epoch 4/20
4982/4982 [==============================] - 22s 4ms/step - loss: 1.7748 - accuracy: 0.4558 - val_loss: 1.8560 - val_accuracy: 0.4428
Epoch 5/20
4982/4982 [==============================] - 22s 4ms/step - loss: 1.6236 - accuracy: 0.4920 - val_loss: 1.6901 - val_accuracy: 0.4608
Epoch 6/20
4982/4982 [==============================] - 22s 4ms/step - loss: 1.4396 - accuracy: 0.5684 - val_loss: 1.5917 - val_accuracy: 0.6068
Epoch 7/20
4982/4982 [==============================] - 22s 4ms/step - loss: 1.324

In [29]:
results = model_rnn.evaluate(test_data, y_test)

print(results)

2246/2246 [==============================] - 0s 185us/step
[1.6228543833123714, 0.6571683287620544]


**El accuracy de la red recurrente es del 65.71% que es ligeramente menor al de la red convolucional pero el tiempo que toma el entrenamiento de esta red es considerablemente mayor a las anteriores**